In [1]:
import pandas as pd
import re 
import numpy
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import * 
from pyspark.sql.functions import length
from pyspark.sql.functions import udf
from pyspark.sql import functions as f
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, when
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer


In [2]:
spark=SparkSession.builder.appName('mmarulandc').getOrCreate()
csv = '/user/mmarulandc/datasets/articles3.csv'
df1 = spark.read.csv(csv).toPandas()
df1.head(2)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
0,None,id,title,publication,author,date,year,month,url,content
1,103459,151908,Alton Sterling’s son: ’Everyone needs to prote...,Guardian,Jessica Glenza,2016-07-13,2016.0,7.0,https://www.theguardian.com/us-news/2016/jul/1...,The son of a Louisiana man whose father was sh...


In [3]:
#Creando un dataframe que contenga los contedidos de las publicaciones hechas y limpiando el contenido de caracteres especiales.
df1['_cleaned'] = [re.sub(r"[^a-zA-Z0-9]+",' ', str(x)) for x in df1['_c9']]
df1.head()


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_cleaned
0,None,id,title,publication,author,date,year,month,url,content,content
1,103459,151908,Alton Sterling’s son: ’Everyone needs to prote...,Guardian,Jessica Glenza,2016-07-13,2016.0,7.0,https://www.theguardian.com/us-news/2016/jul/1...,The son of a Louisiana man whose father was sh...,The son of a Louisiana man whose father was sh...
2,103460,151909,Shakespeare’s first four folios sell at auctio...,Guardian,None,2016-05-25,2016.0,5.0,https://www.theguardian.com/culture/2016/may/2...,Copies of William Shakespeare’s first four boo...,Copies of William Shakespeare s first four boo...
3,103461,151910,My grandmother’s death saved me from a life of...,Guardian,Robert Pendry,2016-10-31,2016.0,10.0,https://www.theguardian.com/commentisfree/2016...,"Debt: $20, 000, Source: College, credit cards,...",Debt 20 000 Source College credit cards Estima...
4,103462,151911,I feared my life lacked meaning. Cancer pushed...,Guardian,Bradford Frost,2016-11-26,2016.0,11.0,https://www.theguardian.com/commentisfree/2016...,"It was late. I was drunk, nearing my 35th birt...",It was late I was drunk nearing my 35th birthd...


In [4]:
#Para verificar que el procesamiento ocurrio correctamente, haremos un word cloud usando la libreria wordcloud para obtener una representacion visual de las palabras más comunes, esta es la clave para enteder la data y que vamos en el buen camino para el entrenamiento de modelos


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
count_vectorizer = CountVectorizer(stop_words='english')
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df1['_cleaned'])

In [6]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA()
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
said percent billion company new year market companies wednesday bank

Topic #1:
says people like just time don years health new know

Topic #2:
team game said time facebook world just news games players

Topic #3:
said police people city told killed according gun officers state

Topic #4:
like just time new people way know life really think

Topic #5:
trump clinton people said campaign party election voters donald political

Topic #6:
trump said president house obama senate republican new administration white

Topic #7:
said government united state president military syria north war states

Topic #8:
new company energy industry said technology power like companies cars

Topic #9:
people health court law state states care federal says government
